### Imports

In [186]:
%%writefile Wikipedia_Scrape.py

print('Importing Packages...')
import requests
import bs4 
from lxml import etree, html
import wikipedia
import pandas as pd
import numpy as np
import datetime
import re
import ast

## Scraping Wiki Infobox
def value_search(text, capture_v1, capture_v1b, capture_v2, append_list):
    if 'Country' in text:
        if '</li>' in text:
            number = len(re.findall(capture_v1, text))
            for capture_no in range(number):
                new_text = str(re.findall(capture_v1, text)[capture_no])
                f = re.findall(capture_v2, new_text)[0]
                if '<br/>' in f:
                    f = f.replace('<br/>', ' ')
                append_list.append(f)
        else:
            f = re.findall(capture_v1b, text)[0]
            if '<br/>' in f:
                f = f.replace('<br/>', ' ')
            append_list.append(f)
    elif '</li>' in text:
        number = len(re.findall(capture_v1, text))
        for capture_no in range(number):
            new_text = str(re.findall(capture_v1, text)[capture_no])
            if 'href' in new_text:
                f = re.findall(capture_v2, new_text)[0]
                if '<br/>' in f:
                    f = f.replace('<br/>', ' ')
                append_list.append(f)
            else:
                f = new_text
                if '<br/>' in f:
                    f = f.replace('<br/>', ' ')
                append_list.append(f)
    else:
        if 'href' in text:
            f = re.findall(capture_v2, text)[0]
            if '<br/>' in f:
                f = f.replace('<br/>', ' ')
            append_list.append(f)
        else:
            f = re.findall(capture_v1b, text)[0]
            if '<br/>' in f:
                f = f.replace('<br/>', ' ')
            append_list.append(f)
    return(append_list)

def date_search(text, capture_v1, capture_v2, capture_v3):
    found = 0
    if '</li>' in text:
        number = len(re.findall(capture_v1, text))
        for capture_no in range(number):
            new_text = str(re.findall(capture_v1, text)[capture_no])
            if 'United States' in new_text or 'Film Festival' in new_text:
                f = re.findall(capture_v2, new_text)[0]
                if '<br/>' in f:
                    f = f.replace('<br/>', ' ')
                found = 1
            elif found == 0:
                f = re.findall(capture_v2, new_text)[0]
                if '<br/>' in f:
                    f = f.replace('<br/>', ' ')
    return(f)

def time_search(text, capture_v1):
    f = re.findall(capture_v1, text)[0]
    return(f)

def wiki_data_scrape(movie_name, soup, movie_url):

    # getting infobox
    infobox = soup.find('table', {'class': 'infobox'})

    #regex for value_captures
    capture_v1 = r'<li>(.*?)</li>' #no linked value
    capture_v1b= r'<td>(.*?)</td>' #no linked value
    capture_v2 = r'href=\".*?\" title=.*?>(.*?)</a>' #linked value
    capture_v3 = r'([0-9]{4}-[0-9]{2}-[0-9]{2})' #date 2020-12-25
    capture_v3b = r'(W* [0-9]{1,2}, [0-9]{4})' #date v2 April 13, 2012
    capture_v4 = r'([0-9]*) minute' #run time
    capture_v5 = r'<td>(.*?)<' #country/language
    capture_v6 = r'(.*?)<' #country with multiples
    capture_v7 = r'<td>(.*lion?)<'


    #search variables/lists
    directors   = []
    producers   = []
    screen_play = []
    based_on    = 0
    starring    = []
    cinemat     = []
    production  = []
    distribute  = []
    release_date = pd.datetime(2099, 1, 1)
    run_time    = -1
    country     = []
    language    = []
    budget      = np.nan
    box_office  = np.nan

    #loop through infobox on wiki
    for i in range(0, len(infobox.find_all('tr'))):
        text = str(infobox.find_all('tr')[i])

        if 'Directed by' in text:
            try:
                directors = value_search(text, capture_v1, capture_v1b, capture_v2, directors)
            except:
                pass

        elif 'Produced by' in text:
            try:
                producers = value_search(text, capture_v1, capture_v1b, capture_v2, producers)
            except:
                pass

        elif 'Screenplay by' in text or 'Story by' in text or 'Written by' in text:
            try:
                screen_play = value_search(text, capture_v1, capture_v1b, capture_v2, screen_play)
            except:
                pass

        elif 'Based on' in text:
            based_on = 1

        elif 'Starring' in text:
            try:
                starring = value_search(text, capture_v1, capture_v1b, capture_v2, starring)
            except:
                pass

        elif 'Cinematography' in text:
            try:
                cinemat = value_search(text, capture_v1, capture_v1b, capture_v2, cinemat)
            except:
                pass

        elif 'Production' in text:
            try:
                production = value_search(text, capture_v1, capture_v1b, capture_v2, production)
            except:
                pass

        elif 'Distributed by' in text:
            try:
                distribute = value_search(text, capture_v1, capture_v1b, capture_v2, distribute)
            except:
                pass

        elif 'Release date' in text or 'release' in text:
            try:
                release_date = date_search(text, capture_v1, capture_v3, capture_v3b)
            except:
                pass

        elif 'Running time' in text:
            try:
                run_time = time_search(text, capture_v4)
            except:
                pass

        elif 'Country' in text:
            try:
                country = value_search(text, capture_v1, capture_v5, capture_v6, country)
            except:
                pass

        elif 'Language' in text:
            try:
                language = value_search(text, capture_v1, capture_v1b, capture_v5, language)
            except:
                pass

        elif 'Budget' in text:
            temp_list = []
            try:
                budget = value_search(text, capture_v1, capture_v1b, capture_v5, temp_list)[0]
            except:
                pass

        elif 'Box office' in text:
            temp_list = []
            try:
                box_office = value_search(text, capture_v1, capture_v1b, capture_v5, temp_list)[0]
            except:
                pass

    return_list = [movie_name, movie_url, directors, producers, screen_play, based_on, starring, cinemat, production, distribute,
           release_date, run_time, country, language, budget, box_office]
    return(return_list)

def scrape_wiki(df):
    movie_list = ['Charlie and the chocolate factory', 'home alone', 'the little mermaid']

    new_df      = df[['Date Seen', 'Title', 'Zach Rating', 'Hayley Rating']].copy()
    temp_df = pd.DataFrame(columns=['Title', 'Movie_URL', 'Directors','Producers','Story_and_ScreenPlay_Writers','BasedOn_Ind',
                                    'Starring','Cinematographers','ProductionCompany','DistributionCompany',
                                    'ReleaseDate','RunTime_Min','Country','Language','Budget','BoxOffice'
                                    ])
    no_matches = []

    #loop through movies
    for movie_name in df.Title:
        if movie_name in temp_df.Title:
            continue
        if str(list(df.loc[df.Title == movie_name]['Release'])[0]) == 'nan':
            print('{} - no release year'.format(movie_name))
            no_matches.append(movie_name)
            continue
        else:
            movie_release = str(list(df.loc[df.Title == movie_name]['Release'])[0])
            movie_year = datetime.datetime.strptime(movie_release, '%m/%d/%Y').year
            print("\n{} - {}".format(movie_name, movie_year))

            #Easy A Bug
            if movie_name in ['Easy A']:
                movie_name = "\"Easy A\""

            #movie lookup and wiki-find
            page_title = wikipedia.search("{} {} Film".format(str(movie_name), str(movie_year)))[0]
            if '&' in page_title:
                page_title2 = page_title.replace('&', 'and')
            else:
                page_title2 = page_title
            url = (
                'https://en.wikipedia.org/w/api.php?action=query&prop=info&inprop=subjectid&titles={}&format=json'.format(page_title2))
            json_response = requests.get(url).json()
            wiki_page_id  = [page_id for page_id, page_info in json_response['query']['pages'].items()][0]

            #pull wiki page
            movie_title = np.nan
            movie_id    = np.nan

            try:
                movie_title = wikipedia.page(page_title2, auto_suggest=False, redirect=True)
            except:
                movie_id = wikipedia.page(pageid=wiki_page_id)

            try:
                movie_id = wikipedia.page(pageid=wiki_page_id)
            except:
                movie_title = wikipedia.page(page_title2, auto_suggest=False, redirect=True)

            if str(movie_title) != 'nan':
                movie = movie_title
            elif str(movie_id) != 'nan':
                movie = movie_id
            else:
                print("can\'t find url")
                continue

            movie_url = movie.url
            print(movie_url)

            #scraping

            # sending the request
            response = requests.get(movie_url)

            # parsing the response
            soup = bs4.BeautifulSoup(response.text, 'lxml')
            if 'MOVIE' in str(soup).upper():
                try:
                    output_list = wiki_data_scrape(movie_name, soup, movie_url)
                    temp_df.loc[len(temp_df)] = output_list
                except:
                    continue
            else:
                print('url not for movie!')

    #merge ratings df and scraped df
    new_df = pd.merge(new_df, temp_df, on='Title', how='inner')
    return new_df

Overwriting Wikipedia_Scrape.py
